In [ ]:
import numpy as np
import urllib
import os
import json
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine 
pd.set_option('max_colwidth', 50)

### GAKI NO TSUKAI

#### NTV

In [ ]:
import requests
from bs4 import BeautifulSoup

episode_gaki_ntv_array = []

for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]:
    print(year)
    url = "https://www.ntv.co.jp/gaki/r_backnumber/"+str(year)+".html"
    html = requests.get(url).content
    data = BeautifulSoup(html, 'html.parser')


    greplist = data.find_all("ul", {"class": "oa-txt-lists"} )
    for entry in reversed(greplist):
        grepentry = entry.find_all("li")
        for episode in reversed(grepentry):
            episode_gaki_ntv_array.append([str(year),episode.find("h3").text, episode.find("div").text])
            
url = "https://www.ntv.co.jp/gaki/r_backnumber/index.html"
html = requests.get(url).content
data = BeautifulSoup(html, 'html.parser')


greplist = data.find_all("ul", {"class": "oa-txt-lists"} )
for entry in reversed(greplist):
    grepentry = entry.find_all("li")
    for episode in reversed(grepentry):
        episode_gaki_ntv_array.append(['2023',episode.find("h3").text, episode.find("div").text])
print('2023')

episode_gaki_ntv = pd.DataFrame(episode_gaki_ntv_array, columns=['year','date_ja', 'description_ja']).replace(r'\n',' ', regex=True).replace(r'ガキの使い特別版!大喜利大合戦スペシャル! ','', regex=True)
episode_gaki_ntv['month_ja'] = episode_gaki_ntv['date_ja'].str.split('月|日|放',expand=True)[0]
episode_gaki_ntv['day_ja'] = episode_gaki_ntv['date_ja'].str.split('月|日|放',expand=True)[1]
episode_gaki_ntv['broadcast_type'] = episode_gaki_ntv['date_ja'].str.split('日',expand=True)[1]
episode_gaki_ntv['date'] = pd.to_datetime(episode_gaki_ntv['year'] + '-' + episode_gaki_ntv['month_ja']  + '-' +  episode_gaki_ntv['day_ja']).dt.strftime('%Y-%m-%d')
episode_gaki_ntv['description_ja'] = episode_gaki_ntv['description_ja'].str.strip()
print(episode_gaki_ntv['broadcast_type'].unique())
episode_gaki_ntv = episode_gaki_ntv[['date','description_ja']]
episode_gaki_ntv['dupe_number'] = episode_gaki_ntv.groupby(['date']).cumcount()+1
episode_gaki_ntv['day_order'] = episode_gaki_ntv['date'] + '-' + episode_gaki_ntv['dupe_number'].astype('str')
episode_gaki_ntv.to_csv('metadata_gakinotsukai_ntv.csv',index = False)
episode_gaki_ntv

In [ ]:
episode_gaki_ntv_dupe = episode_gaki_ntv[episode_gaki_ntv['date'].duplicated(keep=False)].sort_values("date")
#episode_gaki_ntv_dupe.to_csv('dupe_gaki_ntv.csv')
episode_gaki_ntv_dupe

In [ ]:
episode_gaki_ntv[episode_gaki_ntv['date'].duplicated(keep=False)].sort_values("date")

#### Ghia

In [ ]:
episode_gaki_ghia = pd.read_csv('metadata_gakinotsukai_ghia.csv',index_col=[0])
episode_gaki_ghia.rename(columns={'opening_ja':'opening'}, inplace=True)
episode_gaki_ghia['dupe_number'] = episode_gaki_ghia['dupe_number'].fillna(1).astype('int')
episode_gaki_ghia['day_order'] = episode_gaki_ghia['date'].astype('str') + '-' + episode_gaki_ghia['dupe_number'].astype('str')

# episode_gaki_ghia.loc[episode_gaki_ghia['episode'].isna(),'episode'] = '-'
# #episode_gaki_ghia['dupe_number'] = episode_gaki_ghia.groupby(['date']).cumcount()+1
# episode_gaki_ghia['day_order'] = episode_gaki_ghia['date'] + '-' + episode_gaki_ghia['dupe_number'].astype('str')
#episode_gaki_ghia.to_csv('metadata_gakinotsukai_ghia.csv')
episode_gaki_ghia.head()

In [ ]:
episode_gaki_ghia_dupe = episode_gaki_ghia[episode_gaki_ghia['date'].duplicated(keep=False)].sort_values("date")
#episode_gaki_ghia_dupe.to_csv('dupe_gaki_ghia.csv')
episode_gaki_ghia_dupe

In [ ]:
episode_gaki_ntv

In [ ]:
metadata_gakinotsukai = episode_gaki_ghia.merge(episode_gaki_ntv, how='left', on='day_order')
metadata_gakinotsukai.rename(columns={'date_x':'date'},inplace=True)
# metadata_gakinotsukai['opening'] = metadata_gakinotsukai['metadata']
#metadata_gakinotsukai['metadata']= 'Opening: '+metadata_gakinotsukai['metadata']
metadata_gakinotsukai.loc[metadata_gakinotsukai['episode'] == '-','episode'] = 'SP'
metadata_gakinotsukai.loc[metadata_gakinotsukai['episode'].isna(),'episode'] = 'SP'
metadata_gakinotsukai.loc[metadata_gakinotsukai['name_ja'].str.contains('カウントダウンSP'),'drop' ] = True
metadata_gakinotsukai = metadata_gakinotsukai.loc[metadata_gakinotsukai['drop'].isna()]
metadata_gakinotsukai['show_id'] = 'gakinotsukai'
metadata_gakinotsukai['show'] = 'GAKI NO TSUKAI'
metadata_gakinotsukai['show_ja'] = 'ダウンタウンのガキの使いやあらへんで!!'
metadata_gakinotsukai['description_ja'] = metadata_gakinotsukai['description_ja'].fillna(metadata_gakinotsukai['opening'])
metadata_gakinotsukai.drop(columns=['date_y','dupe_number_y','dupe_number_x','drop'],inplace=True)
metadata_gakinotsukai.to_csv('metadata_gakinotsukai.csv',index = False)
metadata_gakinotsukai

### SUBERANAI HANASHI

In [ ]:
url = "https://www.fujitv.co.jp/suberanai/"
html = requests.get(url).content
data = BeautifulSoup(html, 'html.parser')
greplist = data.find_all("dl")

episode_suberanai_hanashi_array = []
for episode in reversed(greplist):
  dt = str(episode.find_all("dt")[0])[8:19].replace('年','-').replace('月','-').replace('日','').replace('（	','').replace('（','').replace('(','').strip()
  #print(dt)
  dt = datetime.strptime(dt, '%Y-%m-%d')
  dd = episode.find_all("dd")[0]
  text = dd.get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  episode_suberanai_hanashi_array.append([dt,text])
  #print(text)
metadata_suberanaihanashi = pd.DataFrame(episode_suberanai_hanashi_array, columns=['date','description_ja']).sort_values('date')
metadata_suberanaihanashi['episode'] = metadata_suberanaihanashi.index + 1
metadata_suberanaihanashi['segment'] = 0
metadata_suberanaihanashi['show_id'] = 'suberanaihanashi'
metadata_suberanaihanashi['show_ja'] = '松本人志のすべらない話'
metadata_suberanaihanashi['show'] = 'SUBERANAI HANASHI'
metadata_suberanaihanashi.to_csv('metadata_suberanaihanashi.csv')
metadata_suberanaihanashi

### KNIGHT SCOOP

In [ ]:
url = urllib.request.urlopen("https://knightscoop.yabai.moe/data/seasons/all.json")
data = json.loads(url.read().decode())
rebuild_of_knightscoop = []

for year in data['seasons']:
    for episode in year['episodes']:
        for count, segment in enumerate(episode['segments']):
            segment['order'] = '#' + str(count+1)
            segment['episode'] = episode
            rebuild_of_knightscoop.append(segment)

metadata_knightscoop = pd.json_normalize(rebuild_of_knightscoop)
metadata_knightscoop['segmentid'] = metadata_knightscoop['episode.date']+ ' - ' + metadata_knightscoop['order'] 
metadata_knightscoop.drop(columns=['episode.segments'], inplace=True)

metadata_knightscoop.rename(columns={'episode.host':'knightscoop_host', 'detective':'knightscoop_detective','location':'knightscoop_location'}, inplace=True)

metadata_knightscoop.loc[metadata_knightscoop['knightscoop_host'].notnull(),'metadata'] = 'ホスト:' + metadata_knightscoop['knightscoop_host'].mask(pd.isnull, '-') + ', 探偵: ' + metadata_knightscoop['knightscoop_detective'].mask(pd.isnull, '-') + ', 位置: ' + metadata_knightscoop['knightscoop_location'].mask(pd.isnull, '-')

metadata_knightscoop['segment'] = metadata_knightscoop['order'].str[1:].astype(int)
metadata_knightscoop['show_id'] = 'knightscoop'
metadata_knightscoop['show_ja'] = '探偵!ナイトスクープ'

metadata_knightscoop['show'] = 'KNIGHT SCOOP'


metadata_knightscoop.rename(columns={'episode.date':'date','episode.number':'episode','topic.jp':'name_ja'}, inplace=True)
metadata_knightscoop['episode'] =pd.factorize(metadata_knightscoop['date'].tolist())[0] +1

metadata_knightscoop = metadata_knightscoop[['show_id','show','show_ja','date','episode','segment','name_ja','metadata','knightscoop_host','knightscoop_detective','knightscoop_location']]

metadata_knightscoop.to_csv('metadata_knightscoop.csv',index = False)

metadata_knightscoop.head()

### VISUALBUM

In [ ]:
metadata_visualbum = pd.read_csv('metadata_visualbum.csv',encoding='utf-8')
metadata_visualbum['segment'] = metadata_visualbum['segment'].astype('int')

metadata_visualbum['show_id'] = 'visualbum'
metadata_visualbum['show'] = 'VISUALBUM'
metadata_visualbum['show_ja']= 'VISUALBUM'
metadata_visualbum

### MHK

In [ ]:
metadata_mhk = pd.read_csv('metadata_mhk.csv',encoding='utf-8')
metadata_mhk['segment'] = metadata_mhk['segment'].astype('int')

metadata_mhk['show_id'] = 'mhk'
metadata_mhk['show'] = 'MHK'
metadata_mhk['show_ja']= '松本一志のコント'
metadata_mhk.drop(columns=['disc','chapter','name_en'],inplace=True)
metadata_mhk

### FREEZE

In [ ]:
# freeze_array = []

# urls = ["https://www.amazon.co.jp/-/en/gp/video/detail/B07GJ93C23/ref=atv_dp_season_select_s1?language=en",
#         "https://www.amazon.co.jp/-/en/gp/video/detail/B06ZYK4X61/ref=atv_dp_season_select_s2?language=en"]

# c = 1
# time.sleep(3)
# print(url)
# html = requests.get("https://www.amazon.co.jp/-/en/gp/video/detail/B07GJ93C23/ref=atv_dp_season_select_s1?language=en").content
# data = BeautifulSoup(html, 'html.parser')
# greplist = data.find_all("ol")[0]
# greplistitem = greplist.find_all("li")

# for episode in greplistitem:
#     name = episode.find_all("span",attrs={"dir":"auto"})[0].text
    
#     description = episode.find_all("div",attrs={"dir":"auto"})[1].text
    
#     metadata = episode.find_all("div",attrs={"class":"_3rBDNv _1AeKJC"})[0]
#     metadata_field = metadata.find_all("div")
#     date = metadata_field[0].text
#     duration = metadata_field[1].text
    
#     freeze_array.append([date, name, description, c])

# c = 2
# time.sleep(3)
# print(url)
# html = requests.get("https://www.amazon.co.jp/-/en/gp/video/detail/B06ZYK4X61/ref=atv_dp_season_select_s2?language=en").content
# data = BeautifulSoup(html, 'html.parser')

# greplist = data.find_all("ol")[0]
# greplistitem = greplist.find_all("li")

# for episode in greplistitem:
#     name = episode.find_all("span",attrs={"dir":"auto"})[0].text

#     description = episode.find_all("div",attrs={"dir":"auto"})[2].text

#     metadata = episode.find_all("div",attrs={"class":"_3rBDNv _1AeKJC"})[0]
#     metadata_field = metadata.find_all("div")
#     date = metadata_field[0].text
#     duration = metadata_field[1].text

#     freeze_array.append([date, name, description, c])

# metadata_freeze = pd.DataFrame(freeze_array[::1], columns=['date','name', 'description_ja','season']).replace(r'\n',' ', regex=True)


metadata_freeze= pd.read_csv('metadata_freeze.csv')
metadata_freeze['name'] = metadata_freeze['name']
metadata_freeze['episode'] = 'S0' + metadata_freeze['season'].astype('str') + 'E0' + metadata_freeze['name'].str[1]
metadata_freeze['date'] = pd.to_datetime(metadata_freeze['date']).dt.strftime('%Y-%m-%d')
metadata_freeze['show_id'] = 'freeze'
metadata_freeze['show'] = 'FREEZE'
metadata_freeze['show_ja'] = 'FREEZE'

# metadata_freeze.to_csv('metadata_freeze.csv')
metadata_freeze

### DOCUMENTARY OF DOCUMENTAL

In [ ]:
# documentaryofdocumental_array = []

# urls = ["https://www.amazon.co.jp/-/en/gp/video/detail/B077BHTVCX/ref=atv_dp_season_select_s1?language=en",
#       "https://www.amazon.co.jp/-/en/dp/B08H5TFTBZ/ref=sr_1_2?crid=ASWOAJ4YSJ6&keywords=documentary+of+documental&qid=1660492151&s=instant-video&sprefix=documentary+of+document%2Cinstant-video%2C196&sr=1-2"]

# c = 0
# for url in urls:
#     c = c+1
#     time.sleep(3)
#     print(url)
#     html = requests.get(url).content
#     data = BeautifulSoup(html, 'html.parser')

#     greplist = data.find_all("ol")[0]
#     greplistitem = greplist.find_all("li")

#     for episode in greplistitem:
#         name = episode.find_all("span",attrs={"dir":"auto"})[0].text
        
#         description = episode.find_all("div",attrs={"dir":"auto"})[1].text
        
#         metadata = episode.find_all("div",attrs={"class":"_3rBDNv _1AeKJC"})[0]
#         metadata_field = metadata.find_all("div")
#         date = metadata_field[0].text
#         duration = metadata_field[1].text
        
#         documentaryofdocumental_array.append([date, name, description, c])

# metadata_documentaryofdocumental = pd.DataFrame(documentaryofdocumental_array[::1], columns=['date','name', 'description_ja','season']).replace(r'\n',' ', regex=True)

In [ ]:
metadata_documentaryofdocumental = pd.read_csv('metadata_documentaryofdocumental.csv',index_col=[0])
metadata_documentaryofdocumental['name'] = metadata_documentaryofdocumental['name']
metadata_documentaryofdocumental['episode'] = 'S0' + metadata_documentaryofdocumental['season'].astype('str') + 'E0' + metadata_documentaryofdocumental['name'].str[1]
metadata_documentaryofdocumental['episode'] = metadata_documentaryofdocumental['episode'].str.replace('S010','S10')
metadata_documentaryofdocumental['date'] = pd.to_datetime(metadata_documentaryofdocumental['date']).dt.strftime('%Y-%m-%d')
metadata_documentaryofdocumental['show_id'] = 'documentaryofdocumental'
metadata_documentaryofdocumental['show'] = 'DOCUMENTARY OF DOCUMENTAL'
metadata_documentaryofdocumental['show_ja'] = 'DOCUMENTARY OF DOCUMENTAL'

#metadata_documentaryofdocumental.to_csv('metadata_documentaryofdocumental.csv')
metadata_documentaryofdocumental

### IKEMENTAL

In [ ]:
# ikemental_array = []

# urls = ["https://www.amazon.co.jp/-/en/dp/B097JZDHSH/ref=sr_1_1?crid=1DS1266GC3WB4&keywords=elemental&qid=1660492482&s=instant-video&sprefix=ikement%2Cinstant-video%2C251&sr=1-1n",]

# c = 0
# for url in urls:
#     c = c+1
#     time.sleep(3)
#     print(url)
#     html = requests.get(url).content
#     data = BeautifulSoup(html, 'html.parser')

#     greplist = data.find_all("ol")[0]
#     greplistitem = greplist.find_all("li")

#     for episode in greplistitem:
#         name = episode.find_all("span",attrs={"dir":"auto"})[0].text
        
#         description = episode.find_all("div",attrs={"dir":"auto"})[0].text
        
#         metadata = episode.find_all("div",attrs={"class":"_3rBDNv _1AeKJC"})[0]
#         metadata_field = metadata.find_all("div")
#         date = metadata_field[0].text
#         duration = metadata_field[1].text
        
#         ikemental_array.append([date, name, description, c])

# metadata_ikemental = pd.DataFrame(ikemental_array[::1], columns=['date','name', 'description_ja','season']).replace(r'\n',' ', regex=True)

metadata_ikemental= pd.read_csv('metadata_ikemental.csv')

metadata_ikemental['name'] = metadata_ikemental['name']
metadata_ikemental['episode'] = 'S0' + metadata_ikemental['season'].astype('str') + 'E0' + metadata_ikemental['name'].str[1]
metadata_ikemental['episode'] = metadata_ikemental['episode'].str.replace('S010','S10')
metadata_ikemental['date'] = pd.to_datetime(metadata_ikemental['date']).dt.strftime('%Y-%m-%d')
metadata_ikemental['show_id'] = 'ikemental'
metadata_ikemental['show'] = 'IKEMENTAL'
metadata_ikemental['show_ja'] = 'イケメンタル'

#metadata_ikemental.to_csv('metadata_ikemental.csv')
metadata_ikemental

### JOSHIMENTAL

In [ ]:
# joshimental_array = []

# urls = ["https://www.amazon.co.jp/-/en/dp/B097JYQC2P/ref=sr_1_2?crid=20113A8JUHYO3&keywords=joshimental&qid=1660492739&s=instant-video&sprefix=joshiment%2Cinstant-video%2C279&sr=1-2",
#         "https://www.amazon.co.jp/-/en/gp/video/detail/B097XHC3CJ/ref=atv_dp_season_select_s2?language=en"]

# c = 0
# for url in urls:
#     c = c+1
#     time.sleep(3)
#     print(url)
#     html = requests.get(url).content
#     data = BeautifulSoup(html, 'html.parser')

#     greplist = data.find_all("ol")[0]
#     greplistitem = greplist.find_all("li")

#     for episode in greplistitem:
#         name = episode.find_all("span",attrs={"dir":"auto"})[0].text
        
#         description = episode.find_all("div",attrs={"dir":"auto"})[0].text
        
#         metadata = episode.find_all("div",attrs={"class":"_3rBDNv _1AeKJC"})[0]
#         metadata_field = metadata.find_all("div")
#         date = metadata_field[0].text
#         duration = metadata_field[1].text
        
#         joshimental_array.append([date, name, description, c])

# metadata_joshimental = pd.DataFrame(joshimental_array[::1], columns=['date','name', 'description_ja','season']).replace(r'\n',' ', regex=True)


metadata_joshimental= pd.read_csv('metadata_joshimental.csv')

metadata_joshimental['name'] = metadata_joshimental['name']
metadata_joshimental['episode'] = 'S0' + metadata_joshimental['season'].astype('str') + 'E0' + metadata_joshimental['name'].str[1]
metadata_joshimental['episode'] = metadata_joshimental['episode'].str.replace('S010','S10')
metadata_joshimental['date'] = pd.to_datetime(metadata_joshimental['date']).dt.strftime('%Y-%m-%d')
metadata_joshimental['show_id'] = 'joshimental'
metadata_joshimental['show'] = 'JOSHIMENTAL'
metadata_joshimental['show_ja'] = '女子メンタル'

#metadata_joshimental.to_csv('metadata_joshimental.csv')
metadata_joshimental

### DOCUMENTAL

In [ ]:
metadata_documental = pd.read_csv('metadata_documental_amazon.csv',index_col=[0])

metadata_documental['name'] = metadata_documental['name']
metadata_documental['episode'] = 'S0' + metadata_documental['season'].astype('str') + 'E0' + metadata_documental['name'].str[1]
metadata_documental['description_ja'] = metadata_documental['description_ja'].str.replace('(C)2019 YD Creation','')
metadata_documental['episode'] = metadata_documental['episode'].str.replace('S010','S10')
metadata_documental['date'] = pd.to_datetime(metadata_documental['date']).dt.strftime('%Y-%m-%d')
metadata_documental['show_id'] = 'documental'
metadata_documental['show'] = 'DOCUMENTAL'
metadata_documental['show_ja'] = 'ドキュメンタル'

#metadata_documental.to_csv('metadata_documental.csv')
metadata_documental

### SUIYOUBI NO DOWNTOWN

In [ ]:
metadata_suiyoubinodowntown = pd.read_csv('https://docs.google.com/spreadsheets/d/1V1vui4S3g1_nJYxuXVc12mH--LZBvC8IvZV5MOW9hwI/gviz/tq?tqx=out:csv&sheet=suiyobi')
#metadata_suiyoubinodowntown.rename(columns={'ccturl':'cct_url'},inplace=True)
metadata_suiyoubinodowntown['episode'] = metadata_suiyoubinodowntown['episode'].fillna(-55).astype(int)
metadata_suiyoubinodowntown['episode'] = metadata_suiyoubinodowntown['episode'].astype('str').str.replace('-55','SP')
#metadata_suiyoubinodowntown['metadata'] = 'Panelists: ' + metadata_suiyoubinodowntown['panelist_1'].mask(pd.isnull,'-') + ', ' + metadata_suiyoubinodowntown['panelist_2'].mask(pd.isnull,'-') + ', ' + metadata_suiyoubinodowntown['panelist_3'].mask(pd.isnull,'-') + ', ' + metadata_suiyoubinodowntown['panelist_4'].mask(pd.isnull,'-')+ ', ' + metadata_suiyoubinodowntown['panelist_5'].mask(pd.isnull,'-')
metadata_suiyoubinodowntown['show_id'] = 'suiyoubinodowntown'
metadata_suiyoubinodowntown['show'] = 'WEDNESDAY DOWNTOWN'
metadata_suiyoubinodowntown['show_ja'] = '水曜日のダウンタウン'
metadata_suiyoubinodowntown.to_csv('metadata_suiyoubinodowntown.csv')
metadata_suiyoubinodowntown

In [ ]:
metadata_suiyoubinodowntown = pd.concat([
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2014')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2015')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2016')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2017')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2018')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2019')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2020')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2021')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2022')[0],
  pd.read_html('https://thetvdb.com/series/wednesdays-downtown/seasons/official/2023')[0]

  ],axis=0)
metadata_suiyoubinodowntown.rename({'Unnamed: 0':'episode','Name':'name_ja','First Aired':'date_and_channel','Runtime':'duration','Image':'image'},axis=1, inplace=True)
metadata_suiyoubinodowntown['date'] = pd.to_datetime(metadata_suiyoubinodowntown['date_and_channel'].str.split(' T',expand=True)[0]).dt.strftime('%Y-%m-%d')
metadata_suiyoubinodowntown = metadata_suiyoubinodowntown[['episode','date','name_ja']]
metadata_suiyoubinodowntown['show_id'] = 'suiyoubinodowntown'
metadata_suiyoubinodowntown['show'] = 'WEDNESDAY DOWNTOWN'
metadata_suiyoubinodowntown['show_ja'] = '水曜日のダウンタウン'
metadata_suiyoubinodowntown.to_csv('metadata_suiyoubinodowntown.csv')
metadata_suiyoubinodowntown

### GOTTSU A KANJI

In [ ]:
metadata_gottsueekanji = pd.read_csv('metadata_gottsueekanji.csv')
metadata_gottsueekanji['date'] = metadata_gottsueekanji['Date'].str.replace('/','-')
metadata_gottsueekanji['episode'] = metadata_gottsueekanji['Episode'].str.replace('#','').str.lstrip('0')
metadata_gottsueekanji['description_ja'] = metadata_gottsueekanji['Content'].str.replace('#','').str.lstrip('0')
metadata_gottsueekanji = metadata_gottsueekanji[['date','episode','description_ja']]
metadata_gottsueekanji['show_id'] = 'gottsueekanji'
metadata_gottsueekanji['show'] = 'GOTTSU A KANJI'
metadata_gottsueekanji['show_ja'] = 'ダウンタウンのごっつええ感じ'
metadata_gottsueekanji

### ASHITA GA ARU SA

In [ ]:
metadata_ashitagaarusa = pd.read_csv('metadata_ashitagaarusa.csv')
metadata_ashitagaarusa['show_id'] = 'ashitagaarusa'
metadata_ashitagaarusa['show'] = 'ASHITA GA ARU SA'
metadata_ashitagaarusa['show_ja'] = '明日があるさ'
metadata_ashitagaarusa

### LINCOLN

In [ ]:
metadata_lincoln = pd.read_csv('metadata_lincoln.csv')[['date','episode']]
metadata_lincoln['show_id'] = 'lincoln'
metadata_lincoln['show'] = 'LINCOLN'
metadata_lincoln['show_ja'] = 'リンカーン'
metadata_lincoln

### WORLD DOWNTOWN

In [ ]:
metadata_worlddowntown = pd.read_csv('metadata_worlddowntown.csv')[['date','episode']]
metadata_worlddowntown['show_id'] = 'worlddowntown'
metadata_worlddowntown['show'] = 'WORLD DOWNTOWN'
metadata_worlddowntown['show_ja'] = 'ワールド・ダウンタウン'
metadata_worlddowntown

## Combination

In [ ]:
cct_index = pd.read_csv('cct_index.csv', index_col = 0)
cct_index = cct_index.sort_values(['cct_id','language'])
cct_index.drop_duplicates(subset='cct_id', keep='first', inplace=True, ignore_index=True)
cct_index['subspath'] = cct_index['cct_id'] + '_en.' + cct_index['sub_format']
cct_index.loc[cct_index['sub_format'].notnull()]
files_uploaded = cct_index.loc[cct_index['uploaded'] == True]
files_uploaded_map_cctid_permalink = dict(zip(files_uploaded['cct_id'].astype('str'),files_uploaded['permalink'].astype('str') ))
files_uploaded_map_cctid_peertubeindex = dict(zip(files_uploaded['cct_id'].astype('str'),files_uploaded['channel'] + '-' + files_uploaded['permalink'].astype('str') ))
files_uploaded_map_cctid_name_en = dict(zip(files_uploaded['cct_id'].astype('str'),files_uploaded['name_en'].astype('str') ))
files_uploaded_map_cctid_subspath = dict(zip(files_uploaded.loc[files_uploaded['subspath'].notna(),'cct_id'].astype('str'),files_uploaded.loc[files_uploaded['subspath'].notna(),'subspath'].astype('str') ))
list(cct_index.columns)

cct_index[['cct_id','permalink','subspath']]

In [ ]:
cct_index.loc[cct_index['subspath'].notnull()]

In [ ]:
metadata_all = pd.concat([
    metadata_gakinotsukai,
    metadata_knightscoop,
    metadata_gottsueekanji,
    metadata_freeze,
    metadata_documental,
    metadata_documentaryofdocumental,
    metadata_ikemental,
    metadata_joshimental,
    metadata_suiyoubinodowntown,
    metadata_ashitagaarusa,
    metadata_mhk,
    metadata_visualbum,
    metadata_lincoln,
    metadata_worlddowntown],ignore_index=True)

metadata_all['segment'] = metadata_all['segment'].fillna(0).astype('int')
metadata_all.loc[(metadata_all['segment'] == 1) & (metadata_all['show_id'] != 'knightscoop')& (metadata_all['show_id'] != 'suiyoubinodowntown'), 'segment'] = 0

metadata_all['cct_id'] = metadata_all['show_id'] +'_'+ metadata_all['date'].str.replace('-','') +'_'+ metadata_all['episode'].astype('str') +'_'+ metadata_all['segment'].astype('str')

metadata_all['name_en'] = metadata_all['cct_id'].map(files_uploaded_map_cctid_name_en)
metadata_all['cct_permalink'] = metadata_all['cct_id'].map(files_uploaded_map_cctid_permalink)
metadata_all['subtitle'] = metadata_all['cct_id'].map(files_uploaded_map_cctid_subspath)

metadata_all.loc[metadata_all['cct_permalink'].str[-2:] == 'en','subbed'] = True
metadata_all.loc[metadata_all['subbed'] != True,'subbed'] = ''

metadata_all.loc[metadata_all['cct_permalink'].notnull(),'cct_url'] = 'https://chikichiki.tube/'+ metadata_all['show_id'] + '/video/' + metadata_all['cct_permalink'].str.lower()

metadata_all.loc[metadata_all['subtitle'] != '','subs_url'] = 'https://raw.githubusercontent.com/chikichikitube/subtitles/main/subtitles/'+ metadata_all['subtitle']

metadata_all.loc[(metadata_all['subtitle'].isna()) & (metadata_all['subbed']== True),'subs_url'] = 'Hardsub'
metadata_all['subs_url'].fillna('',inplace=True)

metadata_all['peertubeindex'] = metadata_all['show_id'] + '-' +  metadata_all['cct_permalink'].str[:-3]

# clean up columns
metadata_db = metadata_all[[ 'cct_id','show_id','show','show_ja','date','episode','segment','subbed','name_ja','name_en','description_ja','metadata','cct_permalink','cct_url','subs_url']]

metadata_db['cct_link'] = '<a href="' + metadata_db['cct_url'] + '">VTR</a>'
# metadata_db['subs_link'] = '<a href="' + metadata_db['subs_url'] + '">Subs</a>'
metadata_db.loc[(metadata_db['subs_url'] != 'Hardsub')& (metadata_db['subbed']== True),'subs_link'] = '<a href="' + metadata_db['subs_url'] + '">Softsub</a>'
metadata_db.loc[(metadata_db['subs_url'] == 'Hardsub'),'subs_link'] = 'Hardsub'

metadata_db.loc[metadata_db['cct_link'].isna(),'cct_link'] = ''
metadata_db.loc[metadata_db['subs_link'].isna(),'subs_link'] = ''

metadata_db['name_ja'].fillna('',inplace=True)
metadata_db['name_en'].fillna('',inplace=True)
metadata_db['name_en'] = metadata_db['name_en'].str.replace('nan','').str.replace('_720p','').str.replace('_1080p','')

metadata_db['description_ja'].fillna('',inplace=True)
metadata_db['metadata'].fillna('',inplace=True)

# cct version (just for videos cct has)
metadata_db['cct_index'] = metadata_db['show_id'] + '-' +  metadata_db['cct_permalink'].str[:-3]
metadata_json = metadata_db.loc[metadata_db['cct_index'].notnull()]
with open('../metadata.json', 'w', encoding='utf-8') as file:
    metadata_json.to_json(file, orient='records', force_ascii=False)
with open('../metadata_index.json', 'w', encoding='utf-8') as file:
    metadata_json.set_index('cct_index').to_json(file, orient='index', force_ascii=False)
metadata_db.to_csv('../metadata.csv',index=False)

metadata_db

In [53]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"../metadata.db")

con = sqlite3.connect('../metadata.db') #not working

metadata_db.to_sql('metadata', con)

2.6.0


8765

In [ ]:
metadata_db.loc[metadata_db['name_en'].isna()]

In [ ]:
metadata_db.loc[metadata_db['name_en'] == 'nan']